In [1]:
import math
import numpy as np
import pandas as pd
import theano
from theano import *
import theano.tensor as T


##set up the dataframe
df= pd.read_csv(r'C:\Users\Dell\Documents\Python Scripts\HW3_data.csv', sep=',',header=None)
df=df.as_matrix(columns=None)
xi = df[0,:]
yi = df[1,:]

#===============#
#cost function  #
#===============#
la1 = theano.shared(value=np.float(-5), name='la1',borrow=True)
la2 = theano.shared(value=np.float(0), name='la2',borrow=True)
x = T.dvector("x")
y = T.dvector("y")

#Construct Theano expression graph
loss = (0.000045*(la2**2)*y - 0.000098*(la1**2)*x + 0.003926*la1*x*T.exp((y**2-x**2)*(la1**2+la2**2)))
cost = T.mean(loss)

train = theano.function(
          inputs=[x,y,la1,la2],
          outputs=[cost],
          name='train',allow_input_downcast=True)

#================#
#cooling schedule#
#================#
tf = lambda t: 0.8*t #temperature function
itf = lambda length: math.ceil(1.2*length) #iteration function

#==============================#
#setting up probability function#
#==============================#
mean = [0,0]
cov = [[1, 0], [0, 1]]

def pfxs(proposal):
    x = proposal[0]
    y = proposal[1]
    new_iter = np.random.multivariate_normal(mean, cov, 1)
    new_x = x + 0.1*new_iter[0][0]
    new_y = y + 0.1*new_iter[0][1]
    return [new_x,new_y]

def simulated_annealing(energyfunc, initials, epochs, tempfunc, iterfunc, proposalfunc,feed_x,feed_y):
    accumulator=[]
    best_solution = old_solution = initials['solution']
    T=initials['T']
    length=initials['length']
    best_energy = old_energy = energyfunc(old_solution[0],old_solution[1],feed_x,feed_y)
    accepted=0
    total=0
    for index in range(epochs):
        print("Epoch", index)
        if index > 0:
            T = tempfunc(T)
            length=iterfunc(length)
        print("Temperature", T, "Length", length)
        
        for it in range(length):
            total+=1
            new_solution = proposalfunc(old_solution)
            new_energy = energyfunc(new_solution[0],new_solution[1])
            
            # Use a min here as you could get a "probability" > 1
            # This is where you decide to jump or not with a probability added 
            alpha = min(1, np.exp((old_energy - new_energy)/T))
                        
            if ((new_energy < old_energy) or (np.random.uniform() < alpha)):
                # Accept proposed solution
                accepted+=1
                accumulator.append((T, new_solution, new_energy))
                if new_energy < best_energy:
                    # Replace previous best with this one
                    best_energy = new_energy
                    best_solution = new_solution
                    best_index=total
                    best_temp=T
                old_energy = new_energy
                old_solution = new_solution
            else:
                # Keep the old stuff
                accumulator.append((T, old_solution, old_energy))
    
    best_meta=dict(index=best_index, temp=best_temp)
    print("frac accepted", accepted/total, "total iterations", total, 'bmeta', best_meta)
    return best_meta, best_solution, best_energy, accumulator


#===============#
#initializing parameters  #
#==============#
inits=dict(solution=[-5,0], length=500, T=10)
bmeta, bs, be, out = simulated_annealing(f, inits, 30, tf, itf, pf)


NameError: name 'pd' is not defined